## Advanced feature engineering, ensembles, and robust evaluation
- Leverage SHAP insights: add radius interactions and physically motivated ratios.
- Explore FFT features (stub to plug in per-target FFT summary if available).
- Optional PCA to reduce noise and collinearity.
- Add stacking ensemble with RandomOverSampler to address class imbalance.
- Add randomized hyperparameter search for GradientBoosting.
- Integrate MC-robust evaluation end-to-end.
- 

# AstroNet-inspired classifier on curated exoplanet dataset
This notebook trains an MLP with dropout (AstroNet-style regularization) to classify exoplanet candidates using curated tabular features derived from light-curve meta-features and stellar/planetary parameters.

Notes from literature:
- Shallue & Vanderburg (2018) popularized AstroNet; dropout and data augmentation help generalization.
- TESS/Kepler follow-ups advocate avoiding leakage (e.g., labels/flags not available at inference).
- Impact parameter and duration/period features are physically informative; uncertainty should be leveraged via Monte Carlo aggregation.

In [9]:
# Imports and setup
import json
import numpy as np
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    average_precision_score,
    confusion_matrix,
    precision_recall_curve,
)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.base import clone
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import RandomOverSampler

# Paths
project_root = Path.cwd()
data_path = project_root.parent / "data" / "unified_exoplanets_final_imputed.csv"
artifacts_dir = project_root / "artifacts"

print("Project root:", project_root)
print("Data path:", data_path)
print("Artifacts dir:", artifacts_dir)

Project root: b:\Akshat\GIT and projects\NASA_SapceApps\Model
Data path: b:\Akshat\GIT and projects\NASA_SapceApps\data\unified_exoplanets_final_imputed.csv
Artifacts dir: b:\Akshat\GIT and projects\NASA_SapceApps\Model\artifacts


In [10]:
# Load curated dataset
data_path = Path('..') / 'data' / 'unified_exoplanets_final_imputed.csv'
df = pd.read_csv(data_path)
print(df.shape)
df.head(3)

(18716, 41)


C:\Users\Akshat\AppData\Local\Temp\ipykernel_22908\3443877944.py:3: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


,planet_name,host_star_id,mission,disposition,orbital_period_days,transit_epoch_bjd,transit_duration_hours,transit_depth_ppm,planet_radius_re,equilibrium_temp_k,...,planet_radius_re_imputed,duration_over_period,duration_over_period_clipped,impact_parameter_pred_std,impact_parameter_ci95_low,impact_parameter_ci95_high,impact_parameter_imputed,equilibrium_temp_k_postfill_median,stellar_logg_postfill_median,stellar_teff_k_postfill_median
0,Kepler-227 b,10797460,Kepler,CONFIRMED,9.488036,170.538750,2.9575,616.0,2.26,793.0,...,False,0.012988,0.012988,NaN,NaN,NaN,False,False,False,False
1,Kepler-227 c,10797460,Kepler,CONFIRMED,54.418383,162.513840,4.5070,875.0,2.83,443.0,...,False,0.003451,0.003451,NaN,NaN,NaN,False,False,False,False
2,K00753.01,10811496,Kepler,CANDIDATE,19.899140,175.850252,1.7822,10800.0,14.60,638.0,...,False,0.003732,0.003732,NaN,NaN,NaN,False,False,False,False


In [3]:
df['disposition'].value_counts()

disposition
CANDIDATE         7429
FALSE POSITIVE    5827
CONFIRMED         4113
CP                 679
KP                 565
FA                  98
REFUTED              5
Name: count, dtype: int64

In [11]:
# Prepare target label with expanded binary mapping and filter definitives

def create_binary_labels(labels: pd.Series) -> pd.Series:
    binary_map = {
        'CONFIRMED': 1,          # Positive class
        'FALSE POSITIVE': 0,     # Negative class
        'REFUTED': 0,            # Treat as false positive
        'FA': 0,                 # False alarm as negative
        # Uncertain / not for supervised training
        'CANDIDATE': None,
        'KP': None,
        'CP': None,
    }
    return labels.map(binary_map)

binary_labels = create_binary_labels(df['disposition'])
mask = ~binary_labels.isna()

# Restrict to definitive labels and attach y
df_sup = df.loc[mask].copy()
df_sup['label'] = binary_labels.loc[mask].astype(int)
print('Definitive label class counts:', df_sup['label'].value_counts().to_dict())
print('Filtered shape:', df_sup.shape)


Definitive label class counts: {0: 5930, 1: 4113}
Filtered shape: (10043, 42)


In [12]:
# Column selection: drop identifiers and high-leakage columns
id_cols = ['planet_name','host_star_id','mission','disposition','ra','dec']
ci_cols = [c for c in df_sup.columns if c.endswith('_ci95_low') or c.endswith('_ci95_high')]
# Keep uncertainty stds and imputation flags; they can carry useful signal without leaking labels
drop_cols = list(set(id_cols + ci_cols))
X = df_sup.drop(columns=drop_cols + ['label'], errors='ignore')
y = df_sup['label']
print('Dropped columns:', drop_cols)
print('Feature count:', X.shape[1])

Dropped columns: ['stellar_teff_k_ci95_low', 'stellar_radius_rsun_ci95_high', 'impact_parameter_ci95_high', 'impact_parameter_ci95_low', 'ra', 'host_star_id', 'planet_radius_re_ci95_high', 'stellar_radius_rsun_ci95_low', 'dec', 'planet_radius_re_ci95_low', 'mission', 'planet_name', 'disposition', 'stellar_teff_k_ci95_high']
Feature count: 27


#### Feature engineering block
Creates interaction and log features informed by SHAP (radius family), residual depth, and a placeholder FFT feature hook. Includes a PCA toggle to compress the numeric space.

In [13]:
# Limit engineered features; include acc_grav_stellar_surface (derive from stellar_logg if missing)

def add_engineered_features(df_in: pd.DataFrame) -> pd.DataFrame:
    df_e = df_in.copy()
    # Ensure requested physical feature exists
    if 'acc_grav_stellar_surface' not in df_e.columns and 'stellar_logg' in df_e.columns:
        # Assume stellar_logg in cgs (log10(cm/s^2)); compute g in cm/s^2
        df_e['acc_grav_stellar_surface'] = np.power(10.0, pd.to_numeric(df_e['stellar_logg'], errors='coerce'))
    elif 'acc_grav_stellar_surface' in df_e.columns:
        df_e['acc_grav_stellar_surface'] = pd.to_numeric(df_e['acc_grav_stellar_surface'], errors='coerce').clip(lower=0)

    # Keep a very small, stable set of engineered features
    # 1) duration/period ratio (bounded)
    if 'transit_duration_hours' in df_e.columns and 'orbital_period_days' in df_e.columns:
        with np.errstate(divide='ignore', invalid='ignore'):
            frac = df_e['transit_duration_hours'] / (24.0 * df_e['orbital_period_days'])
            df_e['duration_over_period_fe'] = np.clip(frac.replace([np.inf, -np.inf], np.nan), 0.0, 1.0)
    # 2) log transform for depth (reduce skew)
    if 'transit_depth_ppm' in df_e.columns:
        df_e['log_transit_depth_ppm'] = np.log1p(np.clip(df_e['transit_depth_ppm'], a_min=0, a_max=None))
    return df_e

# Build features then restrict to definitive labels
X_all = add_engineered_features(df)
X = X_all.loc[mask].copy()
y = df_sup['label']

# Drop identifiers and known leakage columns; keep uncertainty stds and imputation flags
id_cols = ['planet_name','host_star_id','mission','disposition','ra','dec']
ci_cols = [c for c in X.columns if c.endswith('_ci95_low') or c.endswith('_ci95_high')]
drop_cols = list(set(id_cols + ci_cols + ['label']))
X = X.drop(columns=[c for c in drop_cols if c in X.columns], errors='ignore')

print('Engineered features kept:', [c for c in ['acc_grav_stellar_surface','duration_over_period_fe','log_transit_depth_ppm'] if c in X.columns])
print('Final feature count:', X.shape[1])


Engineered features kept: ['acc_grav_stellar_surface', 'duration_over_period_fe', 'log_transit_depth_ppm']
Final feature count: 30


In [14]:
# Separate numeric/categorical and build preprocessing over full definitive dataset
numeric_cols = [c for c in X.columns if X[c].dtype.kind in 'bifc']
categorical_cols = [c for c in X.columns if X[c].dtype.kind not in 'bifc']
preprocess = ColumnTransformer([
    ('num', StandardScaler(), numeric_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
], remainder='drop')

# Train/val/test across the entire definitive dataset
aaa = len(y)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
print('Splits (definitive only):', X_train.shape, X_val.shape, X_test.shape)

# Class weights (to address imbalance)
from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weight_dict = {int(k): float(v) for k, v in zip(classes, class_weights)}
class_weight_dict

Splits (definitive only): (7030, 30) (1506, 30) (1507, 30)


{0: 0.8467839074921706, 1: 1.220910038207711}

### Optional: MC robustness evaluation
We can sample uncertainty-aware columns (e.g., impact_parameter) and retrain/evaluate across several draws, reporting mean ± std of metrics.

In [25]:
# Monte Carlo frames loader (expects mc_frames saved or generated upstream).
# If not available, this cell will create simple bootstrapped surrogates as a fallback.

N_MC_EXPECTED = 40
mc_frames = []
try:
    # If upstream saved MC frames to disk as separate CSVs, you could load them here.
    # Placeholder: not implemented; keep mc_frames empty and rely on upstream import below if exists.
    pass
except Exception as e:
    print("MC frame load failed:", e)

# Optional: derive MC frames from upstream utilities in the data notebook result if available in memory.
# Here, we fallback to simple noise injection on columns with *_pred_std present to mimic MC draws.
if not mc_frames:
    df_sup = df.copy()
    std_cols = [c for c in df_sup.columns if c.endswith("_pred_std")]
    base_cols = [c.replace("_pred_std", "") for c in std_cols]
    rng = np.random.default_rng(42)
    for i in range(N_MC_EXPECTED):
        dfi = df_sup.copy()
        for base, stdc in zip(base_cols, std_cols):
            if base in dfi.columns:
                noise = rng.normal(0, dfi[stdc].fillna(0).to_numpy())
                dfi[base] = (dfi[base].astype(float) + noise).astype(float)
        mc_frames.append(dfi)
print(f"Prepared {len(mc_frames)} MC frames for robust evaluation")

Prepared 40 MC frames for robust evaluation


## Save artifacts and metadata

We persist the calibrated model, threshold, preprocessing, and MC-robust metrics for reproducibility.

In [27]:
from joblib import dump
import json

artifacts_dir.mkdir(parents=True, exist_ok=True)
model_path = artifacts_dir / "stacking_calibrated.joblib"
preprocess_path = artifacts_dir / "preprocess.joblib"
meta_path = artifacts_dir / "metadata.json"

# Pick the trained model object available in the notebook
model_to_save = globals().get("calib") or globals().get("stack_pipe") or globals().get("best_model")
if model_to_save is None:
    raise NameError("No trained model found to save. Expected one of 'calib', 'stack_pipe', or 'best_model'.")

# Save model and preprocess
try:
    dump(model_to_save, model_path)
except Exception as e:
    print("Model save failed:", e)
dump(preprocess, preprocess_path)

# Prepare metadata; mc_stack_results may not be computed if MC step was skipped
metadata = {
    "model": "StackingClassifier + ROS (calibrated isotonic)" if "calib" in globals() else type(model_to_save).__name__,
    "threshold": globals().get("BEST_THRESHOLD", 0.5),
    "test_metrics": {"accuracy": acc, "roc_auc": auc, "pr_auc": ap},
    "features": X.columns.tolist(),
}
if "mc_stack_results" in globals():
    metadata["mc_results"] = mc_stack_results

with open(meta_path, "w") as f:
    json.dump(metadata, f, indent=2)

print(f"Saved model to {model_path}")
print(f"Saved preprocessing to {preprocess_path}")
print(f"Saved metadata to {meta_path}")


Saved model to b:\Akshat\GIT and projects\NASA_SapceApps\Model\artifacts\stacking_calibrated.joblib
Saved preprocessing to b:\Akshat\GIT and projects\NASA_SapceApps\Model\artifacts\preprocess.joblib
Saved metadata to b:\Akshat\GIT and projects\NASA_SapceApps\Model\artifacts\metadata.json


## Calibrated Stacking + ROS (final)

We keep only the calibrated StackingClassifier with RandomOverSampler. We'll fit on train, calibrate on validation, pick a threshold, and report test performance. Then we run MC-robust evaluation (30–50 draws) with 95% CIs and simple error analysis. All other modeling blocks were removed to keep this notebook focused and reproducible.

In [22]:
# Build Stacking + ROS pipeline
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import precision_recall_curve
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import StackingClassifier

# Helper: picklable converter to string for categorical columns

def to_str_df(X):
    try:
        return X.astype(str)
    except Exception:
        return np.asarray(X).astype(str)

# Overwrite preprocess to add imputers (fix NaN handling for LinearSVC and others)
num_pipe = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("sc", StandardScaler()),
])
cat_pipe = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("to_str", FunctionTransformer(to_str_df)),
    ("ohe", OneHotEncoder(handle_unknown="ignore")),
])
preprocess = ColumnTransformer([
    ("num", num_pipe, numeric_cols),
    ("cat", cat_pipe, categorical_cols),
], remainder="drop")

rf = RandomForestClassifier(n_estimators=400, max_depth=None, random_state=42, n_jobs=-1)
gb = GradientBoostingClassifier(random_state=42)
svc = LinearSVC(C=0.5, dual=False, max_iter=5000, random_state=42)
base_estimators = [
    ("rf", rf),
    ("gb", gb),
    ("svc", svc),
]
meta = LogisticRegression(max_iter=2000, class_weight="balanced", n_jobs=None)
stack = StackingClassifier(estimators=base_estimators, final_estimator=meta, passthrough=False, n_jobs=-1)

stack_pipe = ImbPipeline(steps=[
    ("prep", preprocess),
    ("ros", RandomOverSampler(random_state=42)),
    ("clf", stack),
])

# Fit on train and evaluate on validation
stack_pipe.fit(X_train, y_train)

# Calibrate on validation
calib = CalibratedClassifierCV(estimator=stack_pipe, method="isotonic", cv="prefit")
calib.fit(X_val, y_val)

# Robust threshold selection on validation
# Prefer sweeping unique predicted probabilities with a mild precision constraint to avoid all-positive thresholds

def select_threshold(y_true, y_prob, min_precision=0.5, min_recall=None):
    y_true = np.asarray(y_true)
    y_prob = np.asarray(y_prob)
    uniq = np.unique(np.concatenate([[0.0, 1.0], y_prob]))
    best_thr = 0.5
    best_f1 = -1.0
    best_pair = (0.0, 0.0)
    for t in uniq:
        y_hat = (y_prob >= t).astype(int)
        tp = np.sum((y_hat == 1) & (y_true == 1))
        fp = np.sum((y_hat == 1) & (y_true == 0))
        fn = np.sum((y_hat == 0) & (y_true == 1))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        if min_precision is not None and precision < min_precision:
            continue
        if min_recall is not None and recall < min_recall:
            continue
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
        if f1 > best_f1 or (np.isclose(f1, best_f1) and recall > best_pair[1]):
            best_f1 = f1
            best_thr = t
            best_pair = (precision, recall)
    if best_f1 >= 0:
        return float(best_thr)
    # Fallback to PR-curve argmax F1 mapping
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    f1s = 2 * precision * recall / (precision + recall + 1e-12)
    idx = int(np.nanargmax(f1s))
    if idx == 0:
        return float(np.min(y_prob))
    if (idx - 1) < len(thresholds):
        return float(thresholds[idx - 1])
    return float(np.median(y_prob))

val_probs = calib.predict_proba(X_val)[:, 1]
BEST_THRESHOLD = select_threshold(y_val, val_probs, min_precision=0.5)
print(f"Selected threshold on validation (robust): {BEST_THRESHOLD:.3f}")

# Evaluate on test
probs_test = calib.predict_proba(X_test)[:, 1]
preds_test = (probs_test >= BEST_THRESHOLD).astype(int)
acc = accuracy_score(y_test, preds_test)
auc = roc_auc_score(y_test, probs_test)
ap = average_precision_score(y_test, probs_test)
print({"accuracy": acc, "roc_auc": auc, "pr_auc": ap})

# Confusion matrix
cm = confusion_matrix(y_test, preds_test)
print("Confusion matrix:\n", cm)


C:\Users\Akshat\AppData\Roaming\Python\Python310\site-packages\sklearn\calibration.py:330: FutureWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


Selected threshold on validation (robust): 0.500
{'accuracy': 0.9296615792966157, 'roc_auc': 0.9746808588130315, 'pr_auc': 0.9558057906650027}
Confusion matrix:
 [[821  69]
 [ 37 580]]


## Monte Carlo robust evaluation with 95% CIs

We now propagate the upstream imputation uncertainty using multiple MC samples of the dataset. We report mean and 95% confidence intervals for accuracy, ROC AUC, and PR AUC at the selected threshold.

In [ ]:
from math import sqrt
from scipy.stats import t

# Ensure we evaluate using the same selected threshold logic

def evaluate_stacking_over_mc(mc_frames, base_model, preprocess, best_threshold, n_draws=40, random_state=42):
    rng = np.random.default_rng(random_state)
    n = min(n_draws, len(mc_frames))
    metrics = []

    # Local binary mapping consistent with training
    binary_map = {
        'CONFIRMED': 1,
        'FALSE POSITIVE': 0,
        'REFUTED': 0,
        'FA': 0,
    }

    for i in range(n):
        df_i = mc_frames[i]
        # Recreate supervised view and engineered features so columns match X.columns
        df_i_sup = df_i[df_i['disposition'].isin(binary_map.keys())].copy()
        df_i_sup['label'] = df_i_sup['disposition'].map(binary_map).astype(int)
        df_i_sup = add_engineered_features(df_i_sup)

        X_i = df_i_sup[X.columns]
        y_i = df_i_sup['label']
        # Split deterministically to align with earlier split by index
        idx = df_i_sup.index
        X_train_i = X_i.loc[idx.intersection(X_train.index)]
        y_train_i = y_i.loc[X_train_i.index]
        X_val_i = X_i.loc[idx.intersection(X_val.index)]
        y_val_i = y_i.loc[X_val_i.index]
        X_test_i = X_i.loc[idx.intersection(X_test.index)]
        y_test_i = y_i.loc[X_test_i.index]

        # Fit base model fresh on each draw to propagate uncertainty into training too
        m = clone(base_model)
        m.fit(X_train_i, y_train_i)
        calib_i = CalibratedClassifierCV(estimator=m, method='isotonic', cv='prefit')
        calib_i.fit(X_val_i, y_val_i)
        probs = calib_i.predict_proba(X_test_i)[:, 1]
        preds = (probs >= best_threshold).astype(int)
        metrics.append({
            'accuracy': accuracy_score(y_test_i, preds),
            'roc_auc': roc_auc_score(y_test_i, probs),
            'pr_auc': average_precision_score(y_test_i, probs),
        })

    # Aggregate with 95% t-intervals
    def mean_ci(vals):
        arr = np.array(vals, dtype=float)
        m = float(np.mean(arr))
        s = float(np.std(arr, ddof=1)) if len(arr) > 1 else 0.0
        ci = t.ppf(0.975, df=max(len(arr)-1,1)) * (s / sqrt(max(len(arr),1))) if len(arr) > 1 else 0.0
        return m, m-ci, m+ci

    accs = [d['accuracy'] for d in metrics]
    rocs = [d['roc_auc'] for d in metrics]
    prs = [d['pr_auc'] for d in metrics]
    results = {
        'n_draws': n,
        'accuracy': {'mean': mean_ci(accs)[0], 'ci95_low': mean_ci(accs)[1], 'ci95_high': mean_ci(accs)[2]},
        'roc_auc': {'mean': mean_ci(rocs)[0], 'ci95_low': mean_ci(rocs)[1], 'ci95_high': mean_ci(rocs)[2]},
        'pr_auc': {'mean': mean_ci(prs)[0], 'ci95_low': mean_ci(prs)[1], 'ci95_high': mean_ci(prs)[2]},
    }
    return results


In [28]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Ensure predictions exist
if 'probs_test' not in globals():
    probs_test = calib.predict_proba(X_test)[:, 1]
if 'BEST_THRESHOLD' not in globals():
    BEST_THRESHOLD = 0.5
preds_test = (probs_test >= BEST_THRESHOLD).astype(int)

precision = precision_score(y_test, preds_test)
recall = recall_score(y_test, preds_test)
f1 = f1_score(y_test, preds_test)

print({'precision': precision, 'recall': recall, 'f1': f1})

{'precision': 0.8936825885978429, 'recall': 0.940032414910859, 'f1': 0.9162717219589257}
